In [ ]:
#pip install scikit-learn 

In [ ]:
#pip install cython 

In [1]:
pip install collageradiomics==0.3.7

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 89.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 147.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
#pip install pandas

In [3]:
#pip install matplotlib

In [4]:
pip install nibabel

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install scikit-image==0.19.0

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 140.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.18.3
    Uninstalling scikit-image-0.18.3:
      Successfully uninstalled scikit-image-0.18.3

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip show collageradiomics

Name: collageradiomics
Version: 0.3.7
Summary: Python implementation of COLLAGE texture features from BRIC & INVENTLAB.
Home-page: https://github.com/ccipd/collageradiomics
Author: ThetaTech
Author-email: robert.toth@thetatech.ai
License: UNKNOWN
Location: /usr/local/lib/python3.10/dist-packages
Requires: mahotas, numpy, scikit-build, scikit-image, scipy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [7]:
#pip install SimpleITK pyradiomics

In [8]:
#pip install mahotas

In [9]:
#pip install scikit-image==0.19.0

In [10]:
import logging
import os
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
import nibabel as nib
import glob
import pandas as pd 
import collageradiomics 
print(collageradiomics.__name__)
print(collageradiomics.__name__)

from skimage.feature import greycomatrix, greycoprops

FORMAT = '%(asctime)-15s %(levelname)s %(funcName)s  %(message)s'
logging.basicConfig(format=FORMAT, level=logging.DEBUG)

collageradiomics
collageradiomics


# Setting file naming and strings

In [22]:
root_path = '/app/Data/_Brain/Radiology/_Adult/_MS/'
mri_data='MSSEG2'
subdir='preprocessed_cimalab'
time_point='time02'
mri_mod='FLAIR'
pipeline='SkullS_BiasCorrect_rz_IntStnd'
mask_str= 'mask'

In [23]:
label_val = 1
# multilabel mask
if label_val == 1:
    name_a = 'lesion'
elif label_val == 2:
    name_a = 'rim1'
elif label_val == 3:
    name_a = 'rim2'
elif label_val == 4:
    name_a = 'periL'
elif label_val == 5:
    name_a = 'dilation1'
elif label_val == 6:
    name_a = 'dilation2'

name_a

'lesion'

# Functions

In [13]:
def getting_images_list(root_path, mri_data,subdir,time_point,mri_mod,pipeline):

    outpath_base= root_path + mri_data + '/Feature_extraction/Radiomics_features/'
    print('main outpath ....... ',outpath_base)
   
    main_path=os.path.join(root_path , mri_data, subdir,'*',time_point, '*'+ mri_mod + '*' + pipeline +'*.nii*')
    print('main_path MRI modality...',main_path)
    imgs = glob.glob(main_path)

    imgs=sorted(imgs)
    print('\n number of MRI scans: {} , for MIR modality: {}'.format(len(imgs), mri_mod))
 
    return outpath_base, main_path, imgs

In [14]:
def getting_multiple_lesion_list(case_path,case_id):
    seg_paths=os.path.join(case_path, case_id + '*segmentations','*reg.nii.gz')
    seg_im=glob.glob(seg_paths)
    seg_im=sorted(seg_im)
    print('seg images list',len(seg_im))
    
    return seg_im

In [15]:
def getting_images_list_from_csv(mri_data,subdir,time_point,mri_mod,pipeline,mask_str):
    
    # read list of patients from a list CSV XLSX

    xlsx_path=root_path+'UCSF/UCSF-PDGM/Metadata/UCSF-Pre-Post-Os.xlsx'
    data_id=pd.read_excel(xlsx_path)
    ID=data_id['UCSF-PDGM_ID'].tolist()

    labels=[]

    for ids in ID:
        mask_file=glob.glob(os.path.join(root_path , mri_data, mri_data + '-PDGM' '/Preprocessed/*/time02/',ids +'*seg_brats.nii.gz'))
        #print(os.path.join(root_path , mri_data, mri_data + '-PDGM' '/Preprocessed/Post-Op/*/time01/',ids +'*seg_brats.nii.gz'))
        labels.append(mask_file[0])
        #print(mask_files)

    labels=sorted(labels)
    len(labels)
    return labels

In [16]:
def compute_collage_stats(collage,collage_output,clg_af,tm_dim):
    angle_output2=np.zeros((tm_dim[0],tm_dim[1],tm_dim[2],13,2))
    print('angle_output2.shape ....', angle_output2.shape)
    
    for a in range(collage_output.shape[4]):
        dom_angle = collage.dominant_angles[:,:,:,a]
        print('dom_angle.shape ....', dom_angle.shape)
        angle_output = collage._calculate_haralick_textures(dom_angle)
        print('angle_output.shape ....', angle_output.shape)
        angle_output2[:,:,:,:,a]=angle_output
        
        
        for f in range(collage_output.shape[3]):
            c = f*4
            if a >= 1:
                c = (13+f)*4

            feature_angle = angle_output[:,:,:,f]

            f_median = np.nanmedian(feature_angle)
            clg_af[c] = f_median

            f_var = np.nanvar(feature_angle)
            clg_af[c+1] = f_var

            feature_flat = feature_angle.reshape(-1)
            f_skew = st.skew(feature_flat, nan_policy='omit')
            f_skew = np.ma.getdata(f_skew)
            clg_af[c+2] = f_skew

            f_kurt = st.kurtosis(feature_flat, nan_policy='omit')
            clg_af[c+3] = f_kurt
        print('-----------------------------------------------')
    return clg_af, feature_angle, angle_output2

In [17]:
def plot_collage_map(mri_image, label, collage, mri_mod):

    # To show lesion mask overlaid on whole MRI
    z=np.where(label==1)
    zz = round(np.mean([np.min(z[2]),np.max(z[2])])) ### choose central slice

    new_image = mri_image[:,:,zz]
    new_label = label[:,:,zz]
    figure = plt.figure(figsize = (10, 10))
    extent = 0, new_image.shape[1], 0, new_image.shape[0]
    plt.imshow(new_image, cmap=plt.cm.gray, interpolation='none', extent=extent)
    plt.imshow(new_label, cmap=plt.cm.jet, alpha=0.5*(new_label>0), interpolation='none', extent=extent)
    plt.title('slide ' + str(zz))
    plt.savefig(output_name +'/plot_mri_mask_'+ id_label +'_'+ name_a +'_'+pipeline +'.png')

    # To show CoLlAGe feature heatmap overlaid on MRI zoomed in to the bounding box
    bb = zz-np.min(z[2])
    # which_angle = 0
    # angle0 = collage.dominant_angles[:,:,:,which_angle]
    # angle_output0 = collage._calculate_haralick_textures(angle0)

    # To show CoLlAGe feature heatmap overlaid on whole MRI
    which_feature = collageradiomics.HaralickFeature.Entropy
    feature_output = collage.get_single_feature_output(which_feature)
    which_angle = 1
    feature_angle_imsize = feature_output[:,:,zz,which_angle]

    figure = plt.figure(figsize = (10, 10))
    extent = 0, new_image.shape[1], 0, mri_image.shape[0]
    plt.imshow(new_image, cmap=plt.cm.gray, extent=extent)
    plt.imshow(feature_angle_imsize, cmap=plt.cm.jet, alpha=0.5, extent=extent)
    plt.title('slide ' + str(zz))
    plt.savefig(output_name +'/'+ mri_mod +'_'+ 'plot_mri_clg_'+ id_label +'_'+ name_a +'_' + pipeline + '.png')

In [18]:
def plot_collage_map_bounding(mri_image, label, angle_output2,  bines, svd_r, onset_sld, mri_mod):

    # To show lesion mask overlaid on whole MRI
    z=np.where(label==1)
    zz = round(np.mean([np.min(z[2]),np.max(z[2])])) ### choose central slice

    new_image = mri_image[:,:,zz]
    new_label = label[:,:,zz]
    # To show CoLlAGe feature heatmap overlaid on MRI zoomed in to the bounding box
    bb = zz-np.min(z[2])+onset_sld
    # which_angle = 0
    # angle0 = collage.dominant_angles[:,:,:,which_angle]
    # angle_output0 = collage._calculate_haralick_textures(angle0)
    which_feature = collageradiomics.HaralickFeature.Entropy
    feature_angle_imbb = angle_output2[:,:,bb,which_feature,1]

    figure = plt.figure(figsize = (10, 10))
    extent = 0, feature_angle_imbb.shape[1], 0, feature_angle_imbb.shape[0]
    min_y_bb = collage.mask_min_y
    max_y_bb = collage.mask_max_y
    min_x_bb = collage.mask_min_x
    max_x_bb = collage.mask_max_x
    plt.imshow(new_image[min_y_bb:max_y_bb,min_x_bb:max_x_bb], cmap=plt.cm.gray, extent=extent)
    plt.imshow(feature_angle_imbb, cmap=plt.cm.jet, alpha=0.5, extent=extent)
    plt.colorbar()
    plt.savefig(output_name +'/'+ mri_mod +'_'+  id_label +'_'+ str(svd_r) + '_' + str(bines)+'_' + name_a + '_'+ pipeline +'.png')
    

# Set Directories 

In [24]:
outpath_base, main_path, imgs=getting_images_list(root_path,mri_data,subdir,time_point,mri_mod,pipeline)
imgs[10:]

main outpath .......  /app/Data/_Brain/Radiology/_Adult/_MS/MSSEG2/Feature_extraction/Radiomics_features/
main_path MRI modality... /app/Data/_Brain/Radiology/_Adult/_MS/MSSEG2/preprocessed_cimalab/*/time02/*FLAIR*SkullS_BiasCorrect_rz_IntStnd*.nii*

 number of MRI scans: 40 , for MIR modality: FLAIR


['/app/Data/_Brain/Radiology/_Adult/_MS/MSSEG2/preprocessed_cimalab/029/time02/029_02_FLAIR_SkullS_BiasCorrect_rz_IntStnd.nii.gz',
 '/app/Data/_Brain/Radiology/_Adult/_MS/MSSEG2/preprocessed_cimalab/030/time02/030_02_FLAIR_SkullS_BiasCorrect_rz_IntStnd.nii.gz',
 '/app/Data/_Brain/Radiology/_Adult/_MS/MSSEG2/preprocessed_cimalab/032/time02/032_02_FLAIR_SkullS_BiasCorrect_rz_IntStnd.nii.gz',
 '/app/Data/_Brain/Radiology/_Adult/_MS/MSSEG2/preprocessed_cimalab/035/time02/035_02_FLAIR_SkullS_BiasCorrect_rz_IntStnd.nii.gz',
 '/app/Data/_Brain/Radiology/_Adult/_MS/MSSEG2/preprocessed_cimalab/037/time02/037_02_FLAIR_SkullS_BiasCorrect_rz_IntStnd.nii.gz',
 '/app/Data/_Brain/Radiology/_Adult/_MS/MSSEG2/preprocessed_cimalab/039/time02/039_02_FLAIR_SkullS_BiasCorrect_rz_IntStnd.nii.gz',
 '/app/Data/_Brain/Radiology/_Adult/_MS/MSSEG2/preprocessed_cimalab/043/time02/043_02_FLAIR_SkullS_BiasCorrect_rz_IntStnd.nii.gz',
 '/app/Data/_Brain/Radiology/_Adult/_MS/MSSEG2/preprocessed_cimalab/047/time02/047_

# run CoLlAGe

In [ ]:
ids = []
ls_stats = []
ids=[]
id_logs=os.path.join(outpath_base , mri_data + '_78.txt')
for i in list(imgs[24:32]):
    ids.append(i)
    nib_mri = nib.load(i)
    mri_im = nib_mri.get_fdata()
    mri_im = np.fliplr(np.rot90(mri_im)) ### correct orientation, makes generating figures easier
    case_path=os.path.dirname(i)
    id_label = os.path.basename(os.path.dirname(os.path.dirname(i)))
    time_point = os.path.basename((os.path.dirname(i)))
    print('id_label (patient ID) ........... ',id_label)
    print('time_point (time follow up) ........... ',time_point)
    
    seg_im=getting_multiple_lesion_list(case_path,id_label)
    
    for lesion_mask in seg_im:
        lesion_id=os.path.basename(lesion_mask)
        lesion_id=lesion_id.split(".", 1)[0]
        print('lesion_id....',lesion_id)
        ids.append(lesion_id + '_' + name_a)
        
        with open(id_logs, 'w') as f:
            for item in ids:
                f.write("%s\n" %item)
        
        nib_mask = nib.load(lesion_mask)
        mask = nib_mask.get_fdata()
        mask = np.fliplr(np.rot90(mask)) 
        
        label = np.zeros((mask.shape))
        if label_val <=3:
            label[mask==label_val] = 1 ### consider only specified label
        elif label_val == 4:
            label[mask==2] = 1 ### consider lesion plus rim1
            label[mask==3] = 1
        elif label_val == 5:
            label[mask>0] = 1 ### consider all the lesion levels

        label[mri_im==0] = 0

        label_vol=np.where(label==1)
        print('label_vol . ..... ',label_vol[0].shape[0])
        slixes=[np.unique(label_vol[dim]).shape[0] for dim in range(3)]
        
        if label_vol[0].shape[0] > 10 and not np.any(np.array(slixes)==1): # excluding lesion spreaded into one single slide
            
            output_name=os.path.join(outpath_base , id_label , time_point ,lesion_id,'collage')
            if not os.path.exists(output_name):
                os.makedirs(output_name)
            
            for svd_r in list([3,5]):
                for bines in list([4,8,16,32,64]):
                    print('output_name folder .... ',output_name)
                    tmp_name =  os.path.join(output_name, mri_mod + '_' + id_label + '_' + str(svd_r) + '_' + str(bines)+ '_' + name_a + '_' + pipeline + '.npy') 
                    print('tmp_name stats .npy .......',tmp_name)
                    tmp_name_angle =  os.path.join(output_name, mri_mod + '_' + id_label + '_'+ str(svd_r) + '_'+ str(bines)+ '_' + name_a + '_' + pipeline + '_angle_output.npy') 
                    print('tmp_name angles output .npy .......',tmp_name_angle)

                    if not os.path.isfile(tmp_name_angle):

                        print('Calculating CoLlAGe for mask:', lesion_mask)
                        print('Calculating CoLlAGe for label:', name_a)
                        print('Calculating CoLlAGe for "n"  angular bins:', bines)

                        collage = collageradiomics.Collage(mri_im, label, svd_radius=svd_r, verbose_logging=True, num_unique_angles=bines)
                        collage_output = collage.execute()

                        clg_af_init = np.zeros(shape=(104,1))
                        tm_dim = np.shape(collage.dominant_angles[:,:,:,0])
                        clg_af, feature_angle, angle_output2 = compute_collage_stats(collage,collage_output,clg_af_init,tm_dim)
                        tmp_stats = np.squeeze(clg_af.T)
                        np.save(tmp_name, tmp_stats)
                        np.save(tmp_name_angle, angle_output2)
                        print('Temp CoLlAGe feature array saved.\n')

                        onset_sld=0 # how many slides above or below middle slide
                        plot_collage_map_bounding(mri_im, label, angle_output2, bines, svd_r, onset_sld, mri_mod)
                        plot_collage_map(mri_im, label, collage, mri_mod)

                    else:
                        print('\nCoLlAGe already exists (npy) or not calculated. Label %d not found in mask for:' %label_val, id_label, lesion_id)
                        print('..............................................................................................................')
                        continue

        else:
            print('\n insuficient voxels for computing CoLlAGe')
            continue


    print('....................................................................................................................')

id_label (patient ID) ...........  068
time_point (time follow up) ...........  time02
seg images list 54
lesion_id.... lesion_010_reg


2024-12-01 15:38:37,662 DEBUG __init__  Collage Module Initialized
2024-12-01 15:38:37,663 DEBUG __init__  Running 3D Collage = True


label_vol . .....  106
output_name folder ....  /app/Data/_Brain/Radiology/_Adult/_MS/MSSEG2/Feature_extraction/Radiomics_features/068/time02/lesion_010_reg/collage
tmp_name stats .npy ....... /app/Data/_Brain/Radiology/_Adult/_MS/MSSEG2/Feature_extraction/Radiomics_features/068/time02/lesion_010_reg/collage/FLAIR_068_3_4_lesion_SkullS_BiasCorrect_rz_IntStnd.npy
tmp_name angles output .npy ....... /app/Data/_Brain/Radiology/_Adult/_MS/MSSEG2/Feature_extraction/Radiomics_features/068/time02/lesion_010_reg/collage/FLAIR_068_3_4_lesion_SkullS_BiasCorrect_rz_IntStnd_angle_output.npy

CoLlAGe already exists (npy) or not calculated. Label 1 not found in mask for: 068 lesion_010_reg
..............................................................................................................
output_name folder ....  /app/Data/_Brain/Radiology/_Adult/_MS/MSSEG2/Feature_extraction/Radiomics_features/068/time02/lesion_010_reg/collage
tmp_name stats .npy ....... /app/Data/_Brain/Radiology/_Adult/

2024-12-01 15:38:38,260 DEBUG execute  Image shape = (290, 200, 331)
2024-12-01 15:38:38,261 DEBUG execute  Mask size = 5x6x8
2024-12-01 15:38:38,262 DEBUG execute  Image shape (cropped and padded) = (11, 12, 10)
2024-12-01 15:38:38,262 DEBUG execute  Note: Dividing image values by 7999.38623046875 to convert to 0-1 range
2024-12-01 15:38:38,263 DEBUG execute  Calculating pixel gradients:
2024-12-01 15:38:38,265 DEBUG execute  Calculating pixel gradients done.
2024-12-01 15:38:38,265 DEBUG execute  Calculating dominant gradient angles using SVD for each image patch of size 3x3
2024-12-01 15:38:38,266 INFO _svd_dominant_angles  Window patch shape for dominant angle calculation = (7, 7, 1)
2024-12-01 15:38:38,286 DEBUG execute  Calculating dominant gradient angles done.
2024-12-01 15:38:38,287 DEBUG execute  Dominant angles shape = (5, 6, 8, 2)
2024-12-01 15:38:38,287 DEBUG execute  Calculating haralick features of angles:
2024-12-01 15:38:38,288 INFO execute  Calculating features for an

angle_output2.shape .... (5, 6, 8, 13, 2)
dom_angle.shape .... (5, 6, 8)
angle_output.shape .... (5, 6, 8, 13)
-----------------------------------------------
dom_angle.shape .... (5, 6, 8)
angle_output.shape .... (5, 6, 8, 13)
-----------------------------------------------
Temp CoLlAGe feature array saved.



...

2024-12-01 15:40:20,555 DEBUG __init__  Collage Module Initialized
2024-12-01 15:40:20,556 DEBUG __init__  Running 3D Collage = True


output_name folder ....  /app/Data/_Brain/Radiology/_Adult/_MS/MSSEG2/Feature_extraction/Radiomics_features/068/time02/lesion_012_reg/collage
tmp_name stats .npy ....... /app/Data/_Brain/Radiology/_Adult/_MS/MSSEG2/Feature_extraction/Radiomics_features/068/time02/lesion_012_reg/collage/FLAIR_068_5_16_lesion_SkullS_BiasCorrect_rz_IntStnd.npy
tmp_name angles output .npy ....... /app/Data/_Brain/Radiology/_Adult/_MS/MSSEG2/Feature_extraction/Radiomics_features/068/time02/lesion_012_reg/collage/FLAIR_068_5_16_lesion_SkullS_BiasCorrect_rz_IntStnd_angle_output.npy
Calculating CoLlAGe for mask: /app/Data/_Brain/Radiology/_Adult/_MS/MSSEG2/preprocessed_cimalab/068/time02/068_segmentations/lesion_012_reg.nii.gz
Calculating CoLlAGe for label: lesion
Calculating CoLlAGe for "n"  angular bins: 16


2024-12-01 15:40:21,163 DEBUG execute  Image shape = (290, 200, 331)
2024-12-01 15:40:21,164 DEBUG execute  Mask size = 15x8x7
2024-12-01 15:40:21,165 DEBUG execute  Image shape (cropped and padded) = (25, 18, 9)
2024-12-01 15:40:21,166 DEBUG execute  Note: Dividing image values by 7177.87353515625 to convert to 0-1 range
2024-12-01 15:40:21,167 DEBUG execute  Calculating pixel gradients:
2024-12-01 15:40:21,168 DEBUG execute  Calculating pixel gradients done.
2024-12-01 15:40:21,169 DEBUG execute  Calculating dominant gradient angles using SVD for each image patch of size 5x5
2024-12-01 15:40:21,169 INFO _svd_dominant_angles  Window patch shape for dominant angle calculation = (11, 11, 1)
2024-12-01 15:40:21,407 DEBUG execute  Calculating dominant gradient angles done.
2024-12-01 15:40:21,408 DEBUG execute  Dominant angles shape = (15, 8, 7, 2)
2024-12-01 15:40:21,409 DEBUG execute  Calculating haralick features of angles:
2024-12-01 15:40:21,410 INFO execute  Calculating features for

angle_output2.shape .... (15, 8, 7, 13, 2)
dom_angle.shape .... (15, 8, 7)
angle_output.shape .... (15, 8, 7, 13)
-----------------------------------------------
dom_angle.shape .... (15, 8, 7)
angle_output.shape .... (15, 8, 7, 13)
-----------------------------------------------
Temp CoLlAGe feature array saved.



2024-12-01 15:40:23,531 DEBUG _get_ticker_locator_formatter  locator: <matplotlib.ticker.AutoLocator object at 0x7f13fd3374c0>


In [ ]:
i

In [ ]:
np_stats = np.squeeze(ls_stats)

h_names = []
for n in range(collage_output.shape[3]):
    h_names.append(collageradiomics.HaralickFeature(n).name)

s_names = ['median', 'var', 'skew', 'kurt']

names = []
for g in range(collage_output.shape[4]):
    for h in h_names:
        for s in s_names:
            names.append(s + '_' + h + '_' + str(g+1))

h_names

In [ ]:
i

# Write dataframe from directories (.npy)

In [ ]:
root_path = '/app/Data/_Brain/Radiology/_Adult/_Glioma/UCSF/'

In [ ]:
mri_data='UCSF-PDGM'
outpath_base= root_path + mri_data + '/Feature_extraction/Radiomics_texture/'
time_point='time01'
print('main outpath ..... ',outpath_base)

In [ ]:
name_a='Collage_Edema'
npy_files = glob.glob(outpath_base +'/*/'+ time_point +'/collage_delta_01/*'+ name_a +'*put.npy')
npy_files=sorted(npy_files)
print('n .. labels ... ',len(npy_files))
#npy_files
#print(outpath_base +'/*/collage/*'+ name_a +'*.npy')

In [ ]:
txt_files= glob.glob(outpath_base +'/*/'+ time_point +'/collage_delta_01/*'+ name_a +'*.txt')
txt_files=sorted(txt_files)
print('n .. txt files ... ',len(txt_files))

# # Single npy feature vector

In [ ]:
df_shape = pd.DataFrame(np.squeeze(feature_mat), index = id_list, columns = names)
df_idx = pd.DataFrame(np.squeeze(id_list), columns=list(['ID']))

outname=outpath_base + mri_data + '_' + name_a + '.xlsx'
print('saved in ...... ', outname)

with pd.ExcelWriter(outname) as writer:
                df_shape.to_excel(writer, sheet_name="FeatureMatrix",index_label='ID')
#                 df_idx.to_excel(writer, sheet_name="df_idx")


print('patients', np.squeeze(feature_mat).shape)

# load full feature map

In [ ]:
id_list=[]
feature_mat=np.empty((0, 104))
id=-1
for it , npy in zip(list(txt_files[0:1]), list(npy_files[0:1])):
    rows=[]
    id=id+1
    case_id = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(it))))
    npy_id = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(npy))))
    print('case_id .....',case_id)
    feat_vect1=np.load(npy)

print(feat_vect1.shape)
angle_output=feat_vect1

In [ ]:
which_feature = collageradiomics.HaralickFeature.Entropy
print('which_feature....',which_feature)
feature_output = angle_output[:,:,30,which_feature]
print('feature_output....',feature_output.shape)
figure = plt.figure(figsize = (5, 5))
plt.imshow(feature_output, cmap=plt.cm.jet)
plt.show()
flattened_array = feature_output.flatten()
filtered_array = flattened_array[~np.isnan(flattened_array)]
hist_values, bin_edges = np.histogram(filtered_array, bins='auto')

# Plot the histogram
plt.bar(bin_edges[:-1], hist_values, width=np.diff(bin_edges), edgecolor='black')
plt.show()

#  stacked npy

In [ ]:
# writing the dataframe

id_list=[]
feature_mat=np.empty((0, 104))
id=-1
for it , npy in zip(txt_files, npy_files):
    rows=[]
    id=id+1
    case_id = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(it))))
    npy_id = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(npy))))
    print('case_id .....',case_id)
    #print('------------BEGINING OF THE TEXT FILE --------------')
    
    with open(it, 'r') as f:
        data = f.read()        
    rows = data.split('\n')
    feat_vect=np.load(npy)
    feat_vect=feat_vect.reshape(len(rows[0:-1]), 104)
    
    print('feat_vect shape',feat_vect.shape)
    print('txt shape',len(rows[0:-1]))
    print('id ......', id)
    #print(rows)
    if rows[0] == case_id:
            print('>>>>>>>>>>>>>>>starting file for the batch<<<<<<<<<<<<<<<<<')
#             
    for cases in rows[0:-1]:
        if cases not in id_list:
            id_list.append(cases)
            my_list = rows
            indices = [index for index in range(len(my_list)) if my_list[index].startswith(cases)]
            feat_vect1=feat_vect[indices]
            print('indices and feat shape',indices,feat_vect1.shape)
            feature_mat=np.vstack((feature_mat,feat_vect1))
            
    print('------------------END TEXT FILE---------------------')
#id_list

In [ ]:
df_shape = pd.DataFrame(np.squeeze(feature_mat), index = id_list, columns = names)
df_idx = pd.DataFrame(np.squeeze(id_list), columns=list(['ID']))
    
with pd.ExcelWriter(outpath_base + mri_data + '_' + name_a + '.xlsx') as writer:
                df_shape.to_excel(writer, sheet_name="FeatureMatrix",index_label='ID')
                df_idx.to_excel(writer, sheet_name="df_idx")

print('saved in ...... ', outpath_base + mri_data + '_' + name_a + '.xlsx')
print('patients', np.squeeze(feature_mat).shape)